In [10]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPoint, Polygon, Point
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import alphashape
import folium
import osmnx as ox
import folium
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union, polygonize
from concave_hull import concave_hull_indexes
import numpy as np

casino_location = (50.63852, 3.059534)

num_mins = 15

G = ox.graph_from_point(casino_location, dist=5000, network_type='drive')
center_node = ox.distance.nearest_nodes(G, casino_location[1], casino_location[0])
subgraph = nx.ego_graph(G, center_node, radius=num_mins, distance='time')
isochrone = ox.graph_to_gdfs(subgraph, edges=False)

In [11]:
points = [Point(lon, lat) for lon, lat in zip(isochrone['geometry'].x, isochrone['geometry'].y)]

In [12]:
import numpy as np

In [13]:
isochrone = ox.graph_to_gdfs(subgraph, edges=False)
isochrone_polygon = isochrone.unary_union.convex_hull

In [16]:
other_points = [(48.85,2.35), (48.86,2.35), (48.87,2.35)]
other_points_geom = [Point(lon, lat) for lat, lon in other_points]

valid_geometries = [geom for geom in other_points_geom if geom.is_valid]
points_within_isochrone = [point for point in valid_geometries if isochrone_polygon.contains(point)]

m = folium.Map(location=casino_location, zoom_start=13)

#folium.GeoJson(Polygon(list(isochrone_polygon.exterior.coords))).add_to(m)

point_array = np.array([[point.x, point.y] for point in points])
idxes = concave_hull_indexes(
    point_array[:, :2],
    #length_threshold=1,
    concavity=0.8,
)

polygon_coords = point_array[idxes, ::-1].tolist()

folium.Polygon(locations=polygon_coords, color='blue', fill=True, fill_color='blue', fill_opacity=0.4).add_to(m)




# Adding markers for the center and points within the concave hull
folium.Marker(location=casino_location, popup='Center Point', icon=folium.Icon(color='red')).add_to(m)
for point in points_within_isochrone:
    folium.Marker(location=(point.y, point.x), popup='Point Within 15 mins', icon=folium.Icon(color='green')).add_to(m)

# Display the map
m

In [5]:
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('my_map.png')